In [8]:
import pandas as pd

data = pd.read_csv('data/MainTable.csv', delimiter=',')

data.head(3)

,Order,SubjectID,ToolInstances,ServerTimestamp,ServerTimezone,CourseID,CourseSectionID,TermID,AssignmentID,ProblemID,...,CodeStateID,IsEventOrderingConsistent,EventType,Score,Compile.Result,CompileMessageType,CompileMessageData,EventID,ParentEventID,SourceLocation
0,0,14502,Java 8; CodeWorkout,2019-03-20T19:34:25,0,CS 1114,410.0,spring-2019,492.0,32,...,1804399,True,Run.Program,1.0,NaN,NaN,NaN,32-84779,NaN,NaN
1,1,14502,Java 8; CodeWorkout,2019-03-20T19:34:25,0,CS 1114,410.0,spring-2019,492.0,32,...,1804399,True,Compile,NaN,Success,NaN,NaN,32-84780,32-84779,NaN
2,2,13504,Java 8; CodeWorkout,2019-03-20T21:02:00,0,CS 1114,407.0,spring-2019,492.0,32,...,1805146,True,Run.Program,1.0,NaN,NaN,NaN,32-84781,NaN,NaN


In [9]:
code_states = pd.read_csv('data/CodeStates.csv', delimiter=',')

code_states.head(3)

,CodeStateID,Code
0,1804399,"public String plusOut(String str, String word)..."
1,1805146,"public String plusOut(String str, String word)..."
2,1805216,"public String plusOut(String str, String word)..."


In [10]:
data = data.loc[data['EventType'] == 'Run.Program']

# Input, -> CodeStates.csv -> get the corresponding Code field for each CodeStateID

renamed_data = pd.DataFrame()
renamed_data[['AnonID', 'Timestamp', 'Assessment', 'ProblemName', 'TransactionID', 'Score', 'CodeStateID']] = data[['SubjectID', 'ServerTimestamp', 'AssignmentID', 'ProblemID', 'EventID', 'Score', 'CodeStateID']]
renamed_data[['SessionID']] = 0
renamed_data[['MaxScore']] = 1
renamed_data = renamed_data.merge(code_states, on='CodeStateID', how='left')

renamed_data.head(3)

,AnonID,Timestamp,Assessment,ProblemName,TransactionID,Score,CodeStateID,SessionID,MaxScore,Code
0,14502,2019-03-20T19:34:25,492.0,32,32-84779,1.0,1804399,0,1,"public String plusOut(String str, String word)..."
1,13504,2019-03-20T21:02:00,492.0,32,32-84781,1.0,1805146,0,1,"public String plusOut(String str, String word)..."
2,13499,2019-03-20T21:13:09,492.0,32,32-84783,0.0,1805216,0,1,"public String plusOut(String str, String word)..."


In [25]:
renamed_data = renamed_data.sort_values(by='Timestamp')
renamed_data['SubmissionNumber'] = renamed_data.groupby(['AnonID', 'ProblemName']).cumcount() + 1



In [27]:
renamed_data.to_csv('data/FilteredMainTable.csv', sep=',', index=False)
